- in our original agenda this week we try different covariance estimation method
- but I think we'd better go directly to regression model
- then we can save some time for further more complicate machine learning models


- this week we focus on the most simple linear regression without regularization

In [1]:
from stock_helper import *
from stock_stats import *
from product_info import *
from imp import reload  
import stock_helper
import stock_stats
reload(stock_helper)
reload(stock_stats)



<module 'stock_stats' from 'C:\\Users\\dell\\stock_stats.py'>

In [2]:
CORE_NUM = int(os.environ['NUMBER_OF_PROCESSORS'])
CORE_NUM

16

In [3]:
import warnings
 
warnings.filterwarnings('ignore')

In [4]:
product_list = ["600519", "000858", "000568", "600809", "002304"]

In [5]:
period = 4096

In [6]:
import os
all_dates = os.listdir(DATA_PATH + product_list[0])
all_dates.sort()
all_dates = np.array(all_dates)
n_days = len(all_dates)


In [7]:
SAVE_PATH = "e:/intern"

In [8]:
import dask
from dask import compute, delayed
import sklearn.preprocessing

In [9]:
os.makedirs(HEAD_PATH+"/all signal", exist_ok=True)
dire_signal_list = ["nr", "dbook", "range.pos", "price.osci", "ma.dif.10", "kdj.k", "kdj.j"]
range_signal_list = ["", "range", "std", "trend.index"]
all_signal_list = np.array([])
for range_signal in range_signal_list:
    for dire_signal in dire_signal_list:
        if len(range_signal)==0:
            signal_name = dire_signal
        else:
            signal_name = dire_signal+"."+range_signal
        all_signal_list = np.append(all_signal_list,signal_name)
signal_list = [signal+".4096" for signal in all_signal_list]
np.append(signal_list, ["ret.4096", "ret.4096.001", "ret.4096.002"])
n_signal = len(signal_list)

- let's begin with week 8

In [10]:
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import lasso_path, LassoCV, RidgeCV
from sklearn.preprocessing import StandardScaler
#y_signal = "ret."+str(period)+".002"





In [11]:
product = product_list[0]
train_mat = load(HEAD_PATH+"/train test mat/"+product+".train.mat.pkl")

- it seems the result is too bad

In [12]:
coef_list=dict([])
y_signal = "ret."+str(period)+".002"
for product in product_list:
    train_mat = load(HEAD_PATH+"/train test mat/"+product+".train.mat.pkl")
    x_train = train_mat.iloc[:, :n_signal].values
    y_train = train_mat.loc[:, y_signal].values
    scaler =  StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    x_train_normal = scaler.transform(x_train)
    y_std = np.std(y_train)
    model = LassoCV(n_alphas=100, fit_intercept=False, cv=5, max_iter=10000).fit(x_train_normal, y_train/y_std)
    coef = model.coef_/x_std*y_std
    print(product)
    print(coef)
    coef_list[product] = coef

# 600276
# [-0.          0.00344643 -0.         -0.         -0.         -0.
#  -0.          0.          0.          0.          0.         -0.
#  -0.          0.         -0.          0.00301317 -0.         -0.
#  -0.         -0.         -0.         -0.          0.         -0.
#  -0.         -0.         -0.00105632 -0.00026428]
# 000538
# [ 0.01266931  0.         -0.          0.          0.          0.
#  -0.00076245  0.          0.         -0.          0.          0.
#   0.         -0.          0.          0.0016023  -0.          0.
#  -0.          0.         -0.00102048  0.          0.         -0.
#   0.          0.          0.00033424 -0.        ]
# 600332
# [-3.0919854e-17  0.0000000e+00 -0.0000000e+00 -0.0000000e+00
#  -0.0000000e+00 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00
#   0.0000000e+00 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00
#  -0.0000000e+00 -0.0000000e+00 -0.0000000e+00  0.0000000e+00
#  -0.0000000e+00 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00
#  -0.0000000e+00 -0.0000000e+00  0.0000000e+00 -0.0000000e+00
#  -0.0000000e+00 -0.0000000e+00 -0.0000000e+00 -0.0000000e+00]
# 600436
# [-0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#   0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -9.08294731e-19 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
# 600535
# [ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
#   0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  9.37426231e-18 -0.00000000e+00
#   0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]




600519
[-0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -1.41676533e-19 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00]
000858
[-0.         -0.005781   -0.         -0.         -0.00643598  0.00046193
  0.          0.         -0.          0.         -0.          0.
  0.0002564   0.00030326 -0.         -0.         -0.         -0.00815474
 -0.          0.         -0.         -0.         -0.0012126  -0.
 -0.00040279 -0.          0.          0.        ]
000568
[-0.         -0.          0.         -0.00023886 -0.05013818  0.
 -0.          0.         -0.         -0.         -0.          0.
  0.         -0.          0.02754591 -0.00589875  0.00791138 -0.
  

In [13]:
SAVE_PATH = "e:/intern"

In [14]:
save(coef_list, SAVE_PATH+"/wine.coef.ret.4096.002.pkl")

In [15]:
coef_list=dict([])
y_signal = "ret."+str(period)+".004"
for product in product_list:
    train_mat = load(HEAD_PATH+"/train test mat/"+product+".train.mat.pkl")
    x_train = train_mat.iloc[:, :n_signal].values
    y_train = train_mat.loc[:, y_signal].values
    scaler =  StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    x_train_normal = scaler.transform(x_train)
    y_std = np.std(y_train)
    model = LassoCV(n_alphas=100, fit_intercept=False, cv=5, max_iter=10000).fit(x_train_normal, y_train/y_std)
    coef = model.coef_/x_std*y_std
    print(product)
    print(coef)
    coef_list[product] = coef
    
# 600519
# [ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
#   0.00000000e+00 -8.45903437e-04 -5.09172095e-06  0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#   0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
#   0.00000000e+00  9.04792620e-02 -0.00000000e+00 -0.00000000e+00
#   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
# 000858
# [ 0.         -0.006704    0.         -0.         -0.06637936  0.00183745
#   0.          0.00623261 -0.         -0.         -0.          0.
#   0.          0.          0.         -0.00558503 -0.         -0.00223805
#  -0.          0.          0.          0.         -0.         -0.00472675
#  -0.         -0.02579366  0.         -0.        ]
# 000568
# [-0.         -0.          0.         -0.         -0.01052026  0.
#  -0.         -0.         -0.         -0.         -0.         -0.00316935
#  -0.         -0.00019701 -0.         -0.00067919 -0.         -0.
#  -0.00235397 -0.         -0.         -0.         -0.          0.
#  -0.         -0.          0.         -0.        ]
# 600809
# [ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
#  -2.02811516e-18 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
#   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
# 002304
# [-0.03718111  0.         -0.         -0.         -0.03664855  0.00117475
#  -0.         -0.         -0.         -0.         -0.         -0.
#   0.         -0.         -0.         -0.00012538 -0.         -0.
#  -0.          0.         -0.         -0.          0.         -0.
#  -0.         -0.          0.         -0.        ]

600519
[ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -8.45903437e-04 -5.09172095e-06  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  9.04792620e-02 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
000858
[ 0.         -0.006704    0.         -0.         -0.06637936  0.00183745
  0.          0.00623261 -0.         -0.         -0.          0.
  0.          0.          0.         -0.00558503 -0.         -0.00223805
 -0.          0.          0.          0.         -0.         -0.00472675
 -0.         -0.02579366  0.         -0.        ]
000568
[-0.         -0.          0.         -0.         -0.01052026  0.
 -0.         -0.         -0.         -0.         -0.         -0.00316935
 -0.         -0.00019701 -0.         -0.00067919 -0

In [16]:
save(coef_list, SAVE_PATH+"/wine.coef.ret.4096.004.pkl")

- we can see that the result is not good

- it's still not good. So we cannot use lasso model

In [17]:
def par_get_signal_mat(file_name, product, signal_list, HEAD_PATH, SAVE_PATH):
    data = load(HEAD_PATH+"/stock pkl/"+product+"/"+file_name)
    signal_mat = functools.reduce(functools.partial(get_signal_mat, product=product, file_name=file_name, HEAD_PATH=SAVE_PATH), signal_list, None)
    save(signal_mat, SAVE_PATH+"/signal mat pkl/"+product+"/"+file_name)

In [18]:
def par_get_daily_pred(file_name, product, coef, strat, HEAD_PATH):
    signal_mat = load(HEAD_PATH+"/signal mat pkl/"+product+"/"+file_name)
    S = np.dot(signal_mat.transpose(),coef)
    save(S, HEAD_PATH+"/tmp pkl/"+product+"/"+strat+"/"+file_name)      



- now let's try lasso regression

In [19]:
strat = "lasso.004."+str(period)
for product in product_list:
    os.makedirs("e:/intern"+"/tmp pkl/"+product+"/"+strat, exist_ok=True)

In [20]:
coef_list = load(SAVE_PATH+"/wine.coef.ret.4096.004.pkl")

In [21]:
for product in product_list:
    os.makedirs("e:/intern"+"/tmp pkl/"+product+"/"+strat, exist_ok=True)

In [22]:
os.makedirs("e:/intern"+"/signal mat pkl", exist_ok=True)
for product in product_list:
    os.makedirs("e:/intern"+"/signal mat pkl/"+product, exist_ok=True)

In [28]:
np.sum(abs(coef_list["600809"])>1e-8)

0

In [40]:
%%time
for product in product_list:
    print(product)
    parLapply(CORE_NUM, all_dates, par_get_signal_mat, product=product, signal_list=signal_list, HEAD_PATH=HEAD_PATH, SAVE_PATH=SAVE_PATH)
    
# 600519
# 000858
# 000568
# 600809
# 002304
# Wall time: 3min 38s









600519
000858
000568
600809
002304
Wall time: 3min 38s


In [31]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        print(product)
        coef = coef_list[product]
        parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
                  product=product, coef=coef, strat=strat, HEAD_PATH="e:/intern")

# 600519
# 000858
# 000568
# 002304
# Wall time: 28.7 s


600519
000858
000568
002304
Wall time: 28.7 s


In [32]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH="e:/intern")

## Wall time: 4.56 s



Wall time: 4.56 s


In [34]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        evaluate_signal("lasso.004", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                            period=4096, split_str="2017",atr_filter=0.02, save_path="signal result atr", reverse=1)

# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 18min 24s



reverse=1
reverse=1
reverse=1
reverse=1
Wall time: 18min 24s


In [36]:
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        signal_stat = load(HEAD_PATH+"/signal result atr/"+product+"."+"lasso.004.4096.trend"+".pkl")
        train_stat = signal_stat["train.stat"]
        good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
        train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        test_stat = signal_stat["test.stat"]
        test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
    
# 600519 train sharpe  nan test sharpe  nan
# 000858 train sharpe  0.38057701510714226 test sharpe  -0.3008558816921885
# 000568 train sharpe  0.2813515578146476 test sharpe  0.03187529849666643
# 002304 train sharpe  0.14356210818780926 test sharpe  -0.6376099873371995


600519 train sharpe  nan test sharpe  nan
000858 train sharpe  0.38057701510714226 test sharpe  -0.3008558816921885
000568 train sharpe  0.2813515578146476 test sharpe  0.03187529849666643
002304 train sharpe  0.14356210818780926 test sharpe  -0.6376099873371995


In [41]:
coef_list = load(SAVE_PATH+"/wine.coef.ret.4096.002.pkl")
strat = "lasso.002."+str(period)
for product in product_list:
    os.makedirs("e:/intern"+"/tmp pkl/"+product+"/"+strat, exist_ok=True)


In [42]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        print(product)
        coef = coef_list[product]
        parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
                  product=product, coef=coef, strat=strat, HEAD_PATH="e:/intern")
        
# 000858
# 000568
# 600809
# 002304
# Wall time: 28.2 s


000858
000568
600809
002304
Wall time: 28.2 s


In [43]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH="e:/intern")

#Wall time: 4.33 s

Wall time: 4.33 s


In [44]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        evaluate_signal("lasso.002", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                            period=4096, split_str="2017",atr_filter=0.02, save_path="signal result atr", reverse=1)
        
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 17min 58s


reverse=1
reverse=1
reverse=1
reverse=1
Wall time: 17min 58s


In [45]:
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        signal_stat = load(HEAD_PATH+"/signal result atr/"+product+"."+"lasso.002.4096.trend"+".pkl")
        train_stat = signal_stat["train.stat"]
        good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
        train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        test_stat = signal_stat["test.stat"]
        test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
        
# 000858 train sharpe  nan test sharpe  nan
# 000568 train sharpe  nan test sharpe  nan
# 600809 train sharpe  nan test sharpe  nan
# 002304 train sharpe  0.3233387998156374 test sharpe  0.42273951806961885


000858 train sharpe  nan test sharpe  nan
000568 train sharpe  nan test sharpe  nan
600809 train sharpe  nan test sharpe  nan
002304 train sharpe  0.3233387998156374 test sharpe  0.42273951806961885


- The result is good
- Now we go to ridge

In [48]:
y_signal = 'ret.4096.004'

In [49]:
coef_list=dict([])

for product in product_list:
    train_mat = load(HEAD_PATH+"/train test mat/"+product+".train.mat.pkl")
    x_train = train_mat.iloc[:, :n_signal].values
    y_train = train_mat.loc[:, y_signal].values
    scaler =  StandardScaler(copy=True, with_mean=True, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    y_std = np.std(y_train)
    x_train_normal = scaler.transform(x_train)
    model = RidgeCV(fit_intercept=False, cv=5).fit(x_train_normal, y_train/y_std)
    coef = model.coef_/x_std*y_std
    print(product)
    print(coef)
    coef_list[product] = coef

# 600519
# [ 1.99528408e-02 -1.14785851e-02 -1.23463858e-03  6.94856700e-04
#  -2.11979375e-01  5.21307515e-03 -2.54808272e-03  6.12308523e-03
#   2.03401416e-03 -9.10236471e-04 -7.27898886e-04 -3.39503947e-05
#  -3.86246526e-04 -8.92302338e-05 -4.45227024e-03 -6.25876719e-03
#   1.39950686e-03  2.64326509e-04  3.50808127e-02 -4.51924017e-04
#   1.90042259e-03  1.23841161e-01  2.13919361e-03 -2.27525340e-02
#   1.10076140e-02 -1.75138258e-02 -1.60660474e-03  3.49551397e-03]
# 000858
# [-0.020389   -0.00056802  0.02200427 -0.00047172 -0.18048022  0.00256965
#   0.00044719  0.08307516  0.01228999  0.00718508 -0.00581727  0.15226992
#  -0.0036148   0.00199131  0.16351221 -0.07051018 -0.08974679 -0.00601154
#  -0.21612421  0.01274563  0.00711811  0.05667203 -0.01741315 -0.03198155
#   0.00935885 -0.31819783  0.00149994 -0.00416277]
# 000568
# [ 1.07586559e-01  8.90084085e-03  7.73062577e-03  1.76866149e-03
#  -2.03793577e-01  2.60423907e-03 -4.06742017e-03  1.76119847e-02
#  -6.21031893e-03  7.73856561e-03 -1.54595721e-04 -8.88364973e-03
#   1.66412162e-03 -3.23295376e-03 -5.97319788e-03 -3.26512260e-02
#   4.05812061e-02 -1.15236581e-02  4.65210070e-02 -3.54357296e-04
#  -1.82129070e-02 -2.06465269e-01 -2.14657418e-03 -7.50873545e-04
#  -5.11414109e-03  1.97682018e-01  5.05289782e-03  3.91067327e-03]
# 600809
# [ 0.02979219  0.00819219 -0.00206064 -0.00626476 -0.03363006  0.0041368
#   0.00167466 -0.0069235  -0.0077763   0.00593895  0.0002517  -0.00168154
#   0.00038114 -0.00136566  0.17225663 -0.00552728 -0.00243395 -0.00517648
#   0.00883877  0.00079726 -0.01134891 -0.18792029  0.01368102  0.00284962
#   0.01334192 -0.02351089  0.00375842 -0.00122849]
# 002304
# [-1.32567230e-01  1.43272854e-02  1.19835161e-02  3.17739979e-03
#  -1.51619916e-01  2.93112195e-03 -2.42614353e-03  1.65047154e-03
#  -8.91141377e-05  3.77997292e-04 -9.04035527e-05  1.02360421e-03
#   1.98379292e-04 -1.73399757e-04 -7.01094835e-04 -5.64838436e-03
#  -9.74883816e-04 -6.61310331e-04  4.11314653e-04  4.38522508e-05
#  -2.77109924e-04  2.00613277e-02  9.06459656e-04 -2.03646653e-03
#  -1.16006363e-03  1.11167671e-01  5.88470127e-03 -6.27657873e-03]


600519
[ 1.99528408e-02 -1.14785851e-02 -1.23463858e-03  6.94856700e-04
 -2.11979375e-01  5.21307515e-03 -2.54808272e-03  6.12308523e-03
  2.03401416e-03 -9.10236471e-04 -7.27898886e-04 -3.39503947e-05
 -3.86246526e-04 -8.92302338e-05 -4.45227024e-03 -6.25876719e-03
  1.39950686e-03  2.64326509e-04  3.50808127e-02 -4.51924017e-04
  1.90042259e-03  1.23841161e-01  2.13919361e-03 -2.27525340e-02
  1.10076140e-02 -1.75138258e-02 -1.60660474e-03  3.49551397e-03]
000858
[-0.020389   -0.00056802  0.02200427 -0.00047172 -0.18048022  0.00256965
  0.00044719  0.08307516  0.01228999  0.00718508 -0.00581727  0.15226992
 -0.0036148   0.00199131  0.16351221 -0.07051018 -0.08974679 -0.00601154
 -0.21612421  0.01274563  0.00711811  0.05667203 -0.01741315 -0.03198155
  0.00935885 -0.31819783  0.00149994 -0.00416277]
000568
[ 1.07586559e-01  8.90084085e-03  7.73062577e-03  1.76866149e-03
 -2.03793577e-01  2.60423907e-03 -4.06742017e-03  1.76119847e-02
 -6.21031893e-03  7.73856561e-03 -1.54595721e-04 -8

In [50]:
save(coef_list, SAVE_PATH+"/wine.coef.ridge.004.pkl")

In [51]:
strat = "ridge.004."+str(period)
for product in product_list:
    os.makedirs("e:/intern"+"/tmp pkl/"+product+"/"+strat, exist_ok=True)
    

In [52]:
%%time
for product in product_list:
    print(product)
    coef = coef_list[product]
    parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
              product=product, coef=coef, strat=strat, HEAD_PATH="e:/intern")

# 600276
# 000538
# 600332
# 600436
# 600535
# Wall time: 25.8 s


600519
000858
000568
600809
002304
Wall time: 35.4 s


In [53]:
%%time
for product in product_list:
    par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH="e:/intern")
    
# Wall time: 2.34 s

Wall time: 5.49 s


In [54]:
%%time
for product in product_list:
    evaluate_signal("ridge.004", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                        period=4096, split_str="2017",atr_filter=0.02, save_path="signal result atr", reverse=1)

# reverse=1
# reverse=1
# reverse=1
# reverse=1
# reverse=1
# Wall time: 22min 20s





reverse=1
reverse=1
reverse=1
reverse=1
reverse=1
Wall time: 22min 20s


In [56]:
for product in product_list:
    signal_stat = load(HEAD_PATH+"/signal result atr/"+product+"."+"ridge.004.4096.trend"+".pkl")
    train_stat = signal_stat["train.stat"]
    good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
    train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    test_stat = signal_stat["test.stat"]
    test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
    print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))
    
# 600519 train sharpe  nan test sharpe  nan
# 000858 train sharpe  0.34907182116984525 test sharpe  0.24989021542014447
# 000568 train sharpe  0.18276877216561643 test sharpe  -0.26042619795187144
# 600809 train sharpe  nan test sharpe  nan
# 002304 train sharpe  0.6001082863621504 test sharpe  -0.22266388236131687




600519 train sharpe  nan test sharpe  nan
000858 train sharpe  0.34907182116984525 test sharpe  0.24989021542014447
000568 train sharpe  0.18276877216561643 test sharpe  -0.26042619795187144
600809 train sharpe  nan test sharpe  nan
002304 train sharpe  0.6001082863621504 test sharpe  -0.22266388236131687


In [57]:
from sklearn.linear_model import ElasticNetCV

In [58]:
strat = "elastic.004."+str(period)

In [59]:
for product in product_list:
    os.makedirs(SAVE_PATH+"/tmp pkl/"+product+"/"+strat, exist_ok=True)

In [60]:
coef_list=dict([])
y_signal= "ret." + str(period)+".004"
for product in product_list:
    train_mat = load(HEAD_PATH+"/train test mat/"+product+".train.mat.pkl")
    x_train = train_mat.iloc[:, :n_signal].values
    y_train = train_mat.loc[:, y_signal].values
    scaler =  StandardScaler(copy=True, with_mean=False, with_std=True)
    scaler.fit(x_train)
    x_std = np.sqrt(scaler.var_)
    y_std = np.std(y_train)
    x_train_normal = scaler.transform(x_train)
    model = ElasticNetCV(n_alphas=100, l1_ratio=0.5, fit_intercept=False, cv=10, max_iter=10000).fit(x_train_normal, y_train/y_std)
    coef = model.coef_/x_std*y_std
    print(product)
    print(coef)
    coef_list[product] = coef

# 600519
# [ 0.         -0.         -0.         -0.         -0.          0.
#  -0.          0.          0.         -0.00068736 -0.         -0.
#  -0.         -0.          0.         -0.          0.         -0.
#   0.          0.          0.          0.06973003 -0.         -0.
#   0.          0.          0.          0.        ]
# 000858
# [ 0.00000000e+00 -5.45108373e-03  0.00000000e+00 -0.00000000e+00
#  -5.83550004e-02  2.02916524e-03  0.00000000e+00  1.72046095e-02
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
#   0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.11333277e-03
#  -0.00000000e+00 -3.80030024e-03 -0.00000000e+00  0.00000000e+00
#   0.00000000e+00  0.00000000e+00 -1.86227455e-05 -5.26501924e-03
#  -0.00000000e+00 -4.55843632e-02  0.00000000e+00 -0.00000000e+00]
# 000568
# [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
#  -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
# 600809
# [ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
#  -5.56393394e-19 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
#  -0.00000000e+00  0.00000000e+00 -0.00000000e+00  0.00000000e+00
#   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
# 002304
# [-0.01113802  0.         -0.         -0.         -0.0244675   0.
#  -0.         -0.         -0.         -0.         -0.         -0.
#  -0.         -0.         -0.         -0.         -0.         -0.
#  -0.         -0.         -0.         -0.          0.         -0.
#  -0.         -0.          0.         -0.        ]

600519
[ 0.         -0.         -0.         -0.         -0.          0.
 -0.          0.          0.         -0.00068736 -0.         -0.
 -0.         -0.          0.         -0.          0.         -0.
  0.          0.          0.          0.06973003 -0.         -0.
  0.          0.          0.          0.        ]
000858
[ 0.00000000e+00 -5.45108373e-03  0.00000000e+00 -0.00000000e+00
 -5.83550004e-02  2.02916524e-03  0.00000000e+00  1.72046095e-02
 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.11333277e-03
 -0.00000000e+00 -3.80030024e-03 -0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.86227455e-05 -5.26501924e-03
 -0.00000000e+00 -4.55843632e-02  0.00000000e+00 -0.00000000e+00]
000568
[-0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.
 -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
600809
[ 0.00000000e+00 -0.00000000e+00 -0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.000000

In [61]:
save(coef_list, SAVE_PATH+"/wine.coef.elastic.004.pkl")

In [62]:
strat = "elastic.004."+str(period)
for product in product_list:
    os.makedirs("e:/intern"+"/tmp pkl/"+product+"/"+strat, exist_ok=True)

In [70]:
%%time
for product in product_list:
    print(product)
    coef = coef_list[product]
    if (np.sum(abs(coef)>1e-8)>0):
        parLapply(CORE_NUM, all_dates, par_get_daily_pred, 
                  product=product, coef=coef, strat=strat, HEAD_PATH="e:/intern")
    
# 600519
# 000858
# 000568
# 600809
# 002304
# Wall time: 21.3 s



600519
000858
000568
600809
002304
Wall time: 21.3 s


In [71]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        par_get_all_signal(strat, all_dates, product, 4096, SAVE_PATH="e:/intern")
    
# Wall time: 3.42 s

Wall time: 3.42 s


In [72]:
%%time
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        evaluate_signal("elastic.004", all_dates, product, 0.001, 20, CORE_NUM, HEAD_PATH, SAVE_PATH,
                            period=4096, split_str="2019-08",atr_filter=0.02, save_path="signal result atr", reverse=1)

    
# reverse=1
# reverse=1
# reverse=1
# Wall time: 15min 26s





reverse=1
reverse=1
reverse=1
Wall time: 15min 26s


In [74]:
for product in product_list:
    if (np.sum(abs(coef_list[product])>1e-8)>-0):
        signal_stat = load(HEAD_PATH+"/signal result atr/"+product+"."+"elastic.004.4096.trend"+".pkl")
        train_stat = signal_stat["train.stat"]
        good_strat = (train_stat["final.result"]["avg.ret"]>0.001)
        train_pnl = train_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        test_stat = signal_stat["test.stat"]
        test_pnl = test_stat["daily.pnl"][:, good_strat].sum(axis=1)/sum(good_strat)
        print(product, "train sharpe ", sharpe(train_pnl), "test sharpe ", sharpe(test_pnl))

# 600519 train sharpe  0.35428100359249076 test sharpe  1.0920916649060126
# 000858 train sharpe  0.6502682398133942 test sharpe  -0.7318608191345223
# 002304 train sharpe  0.19434894619569948 test sharpe  -1.3986880659860967

600519 train sharpe  0.35428100359249076 test sharpe  1.0920916649060126
000858 train sharpe  0.6502682398133942 test sharpe  -0.7318608191345223
002304 train sharpe  0.19434894619569948 test sharpe  -1.3986880659860967


- output to txt file

In [75]:
os.makedirs("d:/intern/input", exist_ok=True)
np.set_printoptions(suppress=True)

In [76]:
for product in product_list:
    file_name = "d:/intern/input/"+product+".signal.txt"
    fo = open(file_name, "w")
    fo.writelines("signal coef\n")
    chosen_signal = coef_list[product]!=0
    for i in np.where(chosen_signal)[0]:
        fo.writelines(signal_list[i]+" "+str(coef_list[product][i])+"\n")
    fo.close()